# Interaction with the World Homework (#3)
Python Computing for Data Science (c) J Bloom, UC Berkeley 2018

Due Tuesday 2pm, Feb 20, 2018

# 1) Monty: The Python Siri

Let's make a Siri-like program (call it Monty!) with the following properties:
   - record your voice command
   - use a webservice to parse that sound file into text
   - based on what the text, take three different types of actions:
       - send an email to yourself
       - do some math
       - tell a joke

So for example, if you say "Monty: email me with subject hello and body goodbye", it will email you with the appropriate subject and body. If you say "Monty: tell me a joke" then it will go to the web and find a joke and print it for you. If you say, "Monty: calculate two times three" it should response with printing the number 6.

Hint: you can use speed-to-text apps like Houndify (or, e.g., Google Speech https://cloud.google.com/speech/) to return the text (but not do the actions). You'll need to sign up for a free API and then follow documentation instructions for using the service within Python. 

Instantiate Monty, give it a list of commands (which are themselves classes)

In [27]:
import os, io
import pyaudio, wave
import creds
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = creds.googlespeech

In [6]:
WAVE_OUTPUT_FILENAME = "new2.wav"
chunk = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
RECORD_SECONDS = 5

def record(WAVE_OUTPUT_FILENAME = "new2.wav", chunk = 1024, 
       FORMAT = pyaudio.paInt16, CHANNELS = 1, RATE = 44100, RECORD_SECONDS = 5):
    p = pyaudio.PyAudio()
    stream = p.open(format = FORMAT, channels = CHANNELS,
                    rate = RATE, input = True,
                    frames_per_buffer = chunk)
    all = []
    for i in range(0, int(RATE / chunk * RECORD_SECONDS)):
        data = stream.read(chunk)
        all.append(data)
    print("* done recording")
    stream.close()
    p.terminate()
    data = b"".join(all)
    with wave.open(WAVE_OUTPUT_FILENAME, "wb") as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(p.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(data)

In [25]:
record()

* done recording


In [ ]:
!ls -lat |head

In [26]:
!open new2.wav

In [28]:
def transcribe_file(speech_file):
    """Transcribe the given audio file."""
    from google.cloud import speech
    from google.cloud.speech import enums
    from google.cloud.speech import types
    client = speech.SpeechClient()

    with io.open(speech_file, 'rb') as audio_file:
        content = audio_file.read()

    audio = types.RecognitionAudio(content=content)
    config = types.RecognitionConfig(
        encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=44100,
        language_code='en-US')

    response = client.recognize(config, audio)
    # Each result is for a consecutive portion of the audio. Iterate through
    # them to get the transcripts for the entire audio file.
    for result in response.results:
        # The first alternative is the most likely one for this portion.
        print('Transcript: {}'.format(result.alternatives[0].transcript))
        
    return response.results

In [29]:
results = transcribe_file(WAVE_OUTPUT_FILENAME)
command = results[0].alternatives[0].transcript.lower()

Transcript: Monte you send an email with subject Foo and body Buzz looks


In [30]:
import smtplib
import os
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email import encoders
from email.utils import COMMASPACE, formatdate

def mail(sender, pwd, to, subject, text, files=[]):
    msg = MIMEMultipart()
    msg["From"] = sender
    msg["To"] = COMMASPACE.join(to)
    msg["Date"] = formatdate(localtime=True)
    msg["Subject"] = subject
    msg.attach(MIMEText(text))
    for file in files:
        part = MIMEBase("application", "octet-stream")
        part.set_payload( open(file,"rb").read() )
        encoders.encode_base64(part)
        part.add_header("Content-Disposition", "attachment; filename='%s'"
                       % os.path.basename(file))
        msg.attach(part)
    # Note that we need to designate the remote SMTP server we want to use.
    mailServer = smtplib.SMTP("smtp.gmail.com", 587)
    mailServer.starttls()
    mailServer.login(sender, pwd)
    mailServer.sendmail(sender, to, msg.as_string())
    mailServer.close()

In [39]:
def tell_joke():
    """Function that gets a joke off the internet and returns it."""
    raise NotImplementedError("TODO")
    
def do_calculation(command):
    """Function that accepts a command to do math and returns the result.
    
    Probably just gonna throw the calculation at google and try to return whatever is in the calculator box."""
    from urllib.parse import urlencode
    import bs4
    import requests
    
    words = command.split()
    the_calculation = ' '.join(words[1:])
    print(the_calculation)
    
    query = {'q':the_calculation}
    url_values = urlencode(query)
    
    url = f"https://www.google.com/search?{url_values}"
    req = requests.get(url)
    #print(req.text)
    soup = bs4.BeautifulSoup(req.text, "lxml")
    print(list(x.get_text() for x in soup.find_all("h2", class_="r")))
    
def send_email(command):
    """Function that accepts a command to send an email with a specifiable subject and body, then does so."""
    words = command.split()
    subject = ""
    try:
        s = words.index("subject")
    except ValueError:
        # no subject specified
        s = -1
    try:
        b = words.index("body")
        body = ' '.join(words[b+1:])
    except ValueError:
        b = -1
        body = ""
    if s >= 0 and b >= 0:
        if words[b] == "and": # remove trailing "and" separating subject and body definitions
            subject = ' '.join(words[s+1:b-1])
        else:
            subject = ' '.join(words[s+1:b-1])
    subject = subject.capitalize()
    body = body.capitalize()
    print(f"Received command\n'{command}'\nSo I will send an email with subject \n{subject}\nand body \n{body}")
    mail(creds.email, creds.pwd, creds.email, subject, body, [])

def act(command):
    """Function that takes the transcript of a command, determines if it is valid, and acts accordingly if so."""
    words = command.split()
    # I decided to just use a simple heuristic to dispatch the 3 actions specified in the problem
    # rather than getting into natural language processing and other heavyweight approaches
    if (words[0] == "Monty") or (words[0] == "Monte"):
        command = ' '.join(words[1:]) # Strip this to leave just the command
        #print("Commands must begin by addresing Monty by name. For example: \"Monty, tell me a joke.\"")
    if "email" in command: # if we see the word email, assume we are to send one
        # putting this first allows the email to contain the words "calculate" or "tell me a joke"...
        print("Gonna send you an email, aw yiss.")
        send_email(command)
    elif "calculate" in command:
        print("Gonna do some math, aw yiss.")
        do_calculation(command)
    elif ("tell" in command and "joke" in command):
        print("Gonna tell you  a joke, aw yiss.")
        tell_joke()
    else:
        print("Didn't understand the command, sorry.")

In [40]:
act(command)

Gonna send you an email, aw yiss.
Received command
'monte you send an email with subject foo and body buzz looks'
So I will send an email with subject 
Foo
and body 
Buzz looks


# 2) Write a program that identifies musical notes from sound (AIFF) files. 

  - Run it on the supplied sound files (12) and report your program’s results. 
  - Use the labeled sounds (4) to make sure it works correctly. The provided sound files contain 1-3 simultaneous notes from different organs.
  - Save copies of any example plots to illustrate how your program works.
  
  https://piazza.com/berkeley/spring2018/ay250class13410/resources -> Homeworks -> hw3_sound_files.zip

Hints: You’ll want to decompose the sound into a frequency power spectrum. Use a Fast Fourier Transform. Be care about “unpacking” the string hexcode into python data structures. The sound files use 32 bit data. Play around with what happens when you convert the string data to other integer sizes, or signed vs unsigned integers. Also, beware of harmonics.

Middle C = 440Hz, recorded as amplitude of some pressure wave. That gets turned into an analog signal which is used to move a speaker. We take the fourier transform and get the "strength" of each frequency. Really we want the power spectrum, which is the real part of the FFT. 

Dominant freq will have the highest amplitude for this problem. Harmonics will have less.
 **Except for the half-harmonic, 220Hz**



use scipy.io, aifc reader built into python, use some kind of dictionary lookup thing

In [ ]:
!ls sound_files

In [ ]:
import aifc
import os, sys
import numpy as np

In [ ]:
if sys.byteorder == "little":
    dt = "<i4"
elif sys.byteorder == "big":
    dt = ">i4"
else:
    raise Error("Can't figure out endianness")

**Questions for Chelsea**

 * What do we do with the channels? If we really keep them separate, we need 2-byte chunks separated by 2 bytes, as each of the channels. How do we do this efficiently?
 * Also, why is the code below not working? Something about dtypes?
 * Once we have the separated channels, do we need to use them both? It should be possible to just analyze one
 * Which FFT function is the most useful/direct to get the power spectrum? scipy.fftpack.fft?

In [ ]:
soundfile = 'sound_files/A4_PopOrgan.aif'
with aifc.open(soundfile) as aiff:
    # what to do with channels?
    params = aiff.getparams()
    data = aiff.readframes(params.nframes)
    databytes = np.fromstring(data, dtype=np.short).byteswap()
    ch1_1 = databytes[0::4]
    ch1_2 = databytes[1::4]
    ch2_1 = databytes[2::4]
    ch2_2 = databytes[3::4]
    
    channel1 = np.zeros(int(len(databytes)/2))
    channel2 = np.zeros(int(len(databytes)/2))
    
    channel1[0::2] = ch1_1
    channel1[1::2] = ch1_2
    channel2[0::2] = ch2_1
    channel2[1::2] = ch2_2
    
    # we get params.sampwidth * params.nchannels bytes per frame, so chunks are sampwidth bytes
    # with sampwidth * (nchannels-1) bytes in between
    print(params, type(data), len(data), type(databytes), len(databytes), f"{len(ints)/params.framerate} secs", databytes[:100], sep="\n")